In [486]:
import pennylane as qml
from pennylane import numpy as np
import pandas as pd
import pickle
import os
import time
from tqdm import tqdm
import concurrent.futures

In [487]:
# definindo o simulador quântico
dev = qml.device("default.qubit", wires=2) 

In [488]:
# Lista para armazenar os ansatze
ansatz_list = []

# Gera 30 ansatz diferentes
for i in range(30):
    def ansatz(params, wires=[0, 1], ansatz_id=i):
        # Aplica um ansatz diferente dependendo do ID
        if ansatz_id == 0:
            qml.RX(params[0], wires=wires[0]),
            qml.RY(params[1], wires=wires[1]),
            qml.CNOT(wires=wires)
        elif ansatz_id == 1:
            qml.RY(params[0], wires=wires[0]),  # RY no qubit medido (cria superposição paramétrica)
            qml.RX(params[1], wires=wires[1]),  # RX no qubit auxiliar
            qml.CNOT(wires=[wires[1], wires[0]]),  # Entrelaçamento controlado pelo qubit 1
            qml.RZ(params[2], wires=wires[0])   # RZ para ajuste fino da fase
        elif ansatz_id == 2:
            qml.CRX(params[0], wires=wires),
            qml.RY(params[1], wires=wires[1]),
            qml.RX(params[2], wires=wires[0])
        elif ansatz_id == 3:
            qml.RY(params[0], wires=wires[0]),
            qml.CRZ(params[1], wires=wires),
            qml.Hadamard(wires=wires[1])
        elif ansatz_id == 4:
            qml.RX(params[0], wires=wires[0]),
            qml.RY(params[1], wires=wires[1]),
            qml.CNOT(wires=wires),
            qml.RZ(params[2], wires=wires[0])
        elif ansatz_id == 5:
            qml.RY(params[0], wires=wires[0]),  # RY em q0 (garante superposição)
            qml.RX(params[1], wires=wires[1]),  # RX em q1
            qml.CNOT(wires=[wires[1], wires[0]]),  # Entrelaça q1 → q0 (controlado por q1)
            qml.RZ(params[2], wires=wires[0])   # Fase em q0 (afeta a medição)
        elif ansatz_id == 6:
            qml.CRY(params[0], wires=wires),
            qml.RZ(params[1], wires=wires[0]),  
            qml.RY(params[2], wires=wires[1])  # Substitui o X fixo
        elif ansatz_id == 7:
            qml.RY(params[0], wires=wires[0]),  # Mudei RZ para RY
            qml.CRX(params[1], wires=wires),
            qml.RX(params[2], wires=wires[1])  # Troquei Hadamard por RX parametrizado
        elif ansatz_id == 8:
            qml.RY(params[0], wires=wires[0]),
            qml.CZ(wires=wires),
            qml.RX(params[1], wires=wires[1]),
            qml.RZ(params[2], wires=wires[0])
        elif ansatz_id == 9:
            qml.CRZ(params[0], wires=wires),
            qml.CNOT(wires=[wires[1], wires[0]]),
            qml.RY(params[1], wires=wires[1])
        elif ansatz_id == 10:
            qml.Hadamard(wires=wires[0]),
            qml.CRX(params[0], wires=wires),
            qml.RY(params[1], wires=wires[1]),
            qml.RZ(params[2], wires=wires[0])
        elif ansatz_id == 11:
            qml.X(wires=wires[0]),
            qml.CRY(params[0], wires=wires),
            qml.Z(wires=wires[1]),
            qml.RX(params[1], wires=wires[0])
        elif ansatz_id == 12:
            qml.RZ(params[0], wires=wires[0]),
            qml.CNOT(wires=wires),
            qml.RX(params[1], wires=wires[1]),
            qml.RY(params[2], wires=wires[0])
        elif ansatz_id == 13:
            qml.Hadamard(wires=wires[0]),
            qml.CZ(wires=wires),
            qml.RY(params[0], wires=wires[1]),
            qml.RZ(params[1], wires=wires[0])
        elif ansatz_id == 14:
            qml.CRX(params[0], wires=wires),
            qml.CNOT(wires=wires),
            qml.RZ(params[1], wires=wires[0]),
            qml.RX(params[2], wires=wires[1])
        elif ansatz_id == 15:
            qml.Y(wires=wires[0]),
            qml.CRZ(params[0], wires=wires),
            qml.Hadamard(wires=wires[1]),
            qml.RY(params[1], wires=wires[0])
        elif ansatz_id == 16:
            qml.RY(params[0], wires=wires[0]),
            qml.CRX(params[1], wires=wires),
            qml.CNOT(wires=wires),
            qml.RZ(params[2], wires=wires[1])
        elif ansatz_id == 17:
            qml.Hadamard(wires=wires[0]),
            qml.CRY(params[0], wires=wires),
            qml.RZ(params[1], wires=wires[1]),
            qml.RX(params[2], wires=wires[0])
        elif ansatz_id == 18:
            qml.X(wires=wires[1]),
            qml.CZ(wires=wires),
            qml.RY(params[0], wires=wires[0]),
            qml.RZ(params[1], wires=wires[1])
        elif ansatz_id == 19:
            qml.RZ(params[0], wires=wires[0]),
            qml.CRY(params[1], wires=wires),
            qml.Hadamard(wires=wires[1]),
            qml.RX(params[2], wires=wires[0])
        elif ansatz_id == 20:
            qml.CNOT(wires=wires),
            qml.CRZ(params[0], wires=wires),
            qml.RX(params[1], wires=wires[0]),
            qml.RY(params[2], wires=wires[1])
        elif ansatz_id == 21:
            qml.Hadamard(wires=wires[0]),
            qml.CRX(params[0], wires=wires),
            qml.CZ(wires=wires),
            qml.RZ(params[1], wires=wires[1])
        elif ansatz_id == 22:
            qml.RY(params[0], wires=wires[0]),
            qml.CNOT(wires=wires),
            qml.CRZ(params[1], wires=wires),
            qml.RX(params[2], wires=wires[1])
        elif ansatz_id == 23:
            qml.Z(wires=wires[0]),
            qml.CRY(params[0], wires=wires),
            qml.RX(params[1], wires=wires[1]),
            qml.RZ(params[2], wires=wires[0])
        elif ansatz_id == 24:
            qml.Hadamard(wires=wires[0]),
            qml.CRZ(params[0], wires=wires),
            qml.RY(params[1], wires=wires[1]),
            qml.RX(params[2], wires=wires[0])
        elif ansatz_id == 25:
            qml.RX(params[0], wires=wires[1]),  # Substitui o X fixo
            qml.CNOT(wires=wires),
            qml.CRX(params[1], wires=wires),
            qml.RZ(params[2], wires=wires[0])
        elif ansatz_id == 26:
            qml.RZ(params[0], wires=wires[0]),
            qml.CZ(wires=wires),
            qml.Hadamard(wires=wires[1]),
            qml.RY(params[1], wires=wires[0])
        elif ansatz_id == 27:
            qml.RY(params[0], wires=wires[0]),
            qml.CRX(params[1], wires=wires),
            qml.Z(wires=wires[1]),
            qml.RX(params[2], wires=wires[0])
        elif ansatz_id == 28:
            qml.Hadamard(wires=wires[0]),
            qml.CNOT(wires=wires),
            qml.CRY(params[0], wires=wires),
            qml.RZ(params[1], wires=wires[1])
        elif ansatz_id == 29:
            qml.RX(params[0], wires=wires[0]),
            qml.CRZ(params[1], wires=wires),
            qml.CNOT(wires=wires),
            qml.RY(params[2], wires=wires[1])

    ansatz_list.append(ansatz)

In [489]:
print(ansatz_list[0])

<function ansatz at 0x7a86eec119e0>


In [490]:
@qml.qnode(dev)
def generic_circuit(params, ansatz_id, x, amplitde_embedding=True):

    """Circuit genérico que aplica um ansatz específico."""
    if amplitde_embedding:
        qml.AmplitudeEmbedding(x, wires=[0, 1], normalize=True, pad_with=0.0)
    else:
        qml.AngleEmbedding(x, wires=[0, 1], rotation="X")

    ansatz = ansatz_list[ansatz_id]
    ansatz(params)
    return qml.expval(qml.PauliZ(0)), qml.expval(qml.PauliZ(1))

## Ansätze com 2 parâmetros: 0, 3, 9, 11, 13, 15, 18, 21, 26, 28
todos os outros possuem 3 parâmetros

In [491]:
# função para fazer a previsão
def predict(params, x, ansatz_id, bias, amplitde_embedding=True):
    """Faz previsões com o circuito genérico."""
    predictions = np.array([generic_circuit(params=params, ansatz_id=ansatz_id, amplitde_embedding=amplitde_embedding, x=x) ])
    predictions = np.mean(predictions)
    return predictions + bias

#função de custo MSE padrão
def MSE(params, X, y, ansatz_id, bias, amplitde_embedding=True):
    """Função de custo para otimização."""
    predictions = np.array([predict(params=params, x=x, ansatz_id=ansatz_id, bias=bias, amplitde_embedding=amplitde_embedding) for x in X])
    return np.mean((predictions - y) ** 2)  # MSE

#função de custo BCE
def BCE(params, X, y, ansatz_id, bias, amplitde_embedding=True):
    """Função de custo para otimização."""
    predictions = np.array([predict(params=params, x=x, ansatz_id=ansatz_id, bias=bias, amplitde_embedding=amplitde_embedding) for x in X])
    predictions = np.clip(predictions, 1e-15, 1 - 1e-15)  # Previne log(0)

    return -np.mean(y * np.log(predictions) + (1 - y) * np.log(1 - predictions))  # BCE

# função para calcular a acurácia
def accuracy(params, X, y, ansatz_id, bias, amplitde_embedding=True):
    """Calcula a acurácia do modelo."""
    predictions = np.array([predict(params=params, x=x, ansatz_id=ansatz_id, bias=bias, amplitde_embedding=amplitde_embedding) for x in X])
    predictions = np.sign(predictions)  # Converte para -1 ou 1
    return np.mean(predictions == y)  # Acurácia

In [ ]:
# lendo os datasets
df1 = pd.read_csv("./../datasets/blobs_datasets/blobs_2classes_2features_50samples_high_noise.csv")
df1.nome = "blobs_2classes_2features_50samples_high_noise"
df2 = pd.read_csv("./../datasets/blobs_datasets/blobs_2classes_2features_50samples_low_noise.csv")
df2.nome = "blobs_2classes_2features_50samples_low_noise"
df3 = pd.read_csv("./../datasets/blobs_datasets/blobs_2classes_2features_50samples_no_noise.csv")
df3.nome = "blobs_2classes_2features_50samples_no_noise"
df4 = pd.read_csv("./../datasets/blobs_datasets/blobs_2classes_2features_500samples_high_noise.csv")
df4.nome = "blobs_2classes_2features_500samples_high_noise"
df5 = pd.read_csv("./../datasets/blobs_datasets/blobs_2classes_2features_500samples_low_noise.csv")
df5.nome = "blobs_2classes_2features_500samples_low_noise"
df6 = pd.read_csv("./../datasets/blobs_datasets/blobs_2classes_2features_500samples_no_noise.csv")
df6.nome = "blobs_2classes_2features_500samples_no_noise"
df7 = pd.read_csv("./../datasets/circles_datasets/circles_50samples_high_noise.csv")
df7.nome = "circles_50samples_high_noise"
df8 = pd.read_csv("./../datasets/circles_datasets/circles_50samples_low_noise.csv")
df8.nome = "circles_50samples_low_noise"
df9 = pd.read_csv("./../datasets/circles_datasets/circles_50samples_no_noise.csv")
df9.nome = "circles_50samples_no_noise"
df10 = pd.read_csv("./../datasets/circles_datasets/circles_500samples_high_noise.csv")
df10.nome = "circles_500samples_high_noise"
df11 = pd.read_csv("./../datasets/circles_datasets/circles_500samples_low_noise.csv")
df11.nome = "circles_500samples_low_noise"
df12 = pd.read_csv("./../datasets/circles_datasets/circles_500samples_no_noise.csv")
df12.nome = "circles_500samples_no_noise"

In [ ]:
# carregando os samples dos dataframes

with open("./../datasets/blobs_metrics/blobs_2classes_2features_50samples_high_noise.pkl", 'rb') as file:
    metrics1 = pickle.load(file)

with open("./../datasets/blobs_metrics/blobs_2classes_2features_50samples_low_noise.pkl", 'rb') as file:
    metrics2 = pickle.load(file)

with open("./../datasets/blobs_metrics/blobs_2classes_2features_50samples_no_noise.pkl", 'rb') as file:
    metrics3 = pickle.load(file)

with open("./../datasets/blobs_metrics/blobs_2classes_2features_500samples_high_noise.pkl", 'rb') as file:
    metrics4 = pickle.load(file)

with open("./../datasets/blobs_metrics/blobs_2classes_2features_500samples_low_noise.pkl", 'rb') as file:
    metrics5 = pickle.load(file)

with open("./../datasets/blobs_metrics/blobs_2classes_2features_500samples_no_noise.pkl", 'rb') as file:
    metrics6 = pickle.load(file)

with open("./../datasets/circles_metrics/circles_50samples_high_noise.pkl", 'rb') as file:
    metrics7 = pickle.load(file)

with open("./../datasets/circles_metrics/circles_50samples_low_noise.pkl", 'rb') as file:
    metrics8 = pickle.load(file)

with open("./../datasets/circles_metrics/circles_50samples_no_noise.pkl", 'rb') as file:
    metrics9 = pickle.load(file)

with open("./../datasets/circles_metrics/circles_500samples_high_noise.pkl", 'rb') as file:
    metrics10 = pickle.load(file)

with open("./../datasets/circles_metrics/circles_500samples_low_noise.pkl", 'rb') as file:
    metrics11 = pickle.load(file)

with open("./../datasets/circles_metrics/circles_500samples_no_noise.pkl", 'rb') as file:
    metrics12 = pickle.load(file)

In [494]:
#settando variáveis para geração de dados

samples = [
    metrics1, metrics2, metrics3,
    metrics4, metrics5, metrics6,
    metrics7, metrics8, metrics9,
    metrics10, metrics11, metrics12
]

data_frames = [
    df1, df2, df3, df4, df5, df6,
    df7, df8, df9, df10, df11, df12
]

cost_functions = [MSE]

optimizers = [qml.AdamOptimizer(stepsize=0.1)]

ansatz_ids = list(range(30))

samples_ids = list(range(30))

In [495]:
def salvar_dataframe(df_novo, arquivo):
    if os.path.exists(arquivo):
        df_existente = pd.read_csv(arquivo)
        df_final = pd.concat([df_existente, df_novo], ignore_index=True)
    else:
        df_final = df_novo
    
    df_final.to_csv(arquivo, index=False)

In [496]:
#treinamento, teste e geração de dados
"""
rng_weights = np.random.default_rng(42)

for ansatz_id in ansatz_ids:
    for (sample, df) in (zip(samples, data_frames)):
        for sample_id in samples_ids:
            for cost_function in (cost_functions):
                for  opt in (optimizers):
                    for amplitde_embedding in ([True]):

                        # resetando o optimizador
                        if opt.__class__.__name__ == "GradientDescentOptimizer":
                            opt = qml.GradientDescentOptimizer(stepsize=0.1)
                        elif opt.__class__.__name__ == "AdamOptimizer":
                            opt = qml.AdamOptimizer(stepsize=0.1)
                        
                        # Definindo os parâmetros atualizaveis pelo otimizador
                        num_params = 3 if ansatz_id not in [0, 3, 9, 11, 13, 15, 18, 21, 26, 28] else 2
                        params = np.array(2 * rng_weights.random(num_params) * np.pi, requires_grad=True)
                        bias = np.array(0.0, requires_grad=True)

                        # Definindo o número de iterações
                        epoch = 20

                        # separando o dataset em treino e teste
                        X_train = df.loc[sample["samples"][sample_id], df.columns != "target"]
                        X_train = X_train.to_numpy()
                        y_train = df.loc[sample["samples"][sample_id], "target"]
                        y_train = y_train.values.flatten()
                        X_test = df.loc[~df.index.isin(sample["samples"][sample_id]), df.columns != "target"]
                        X_test = X_test.to_numpy()
                        y_test = df.loc[~df.index.isin(sample["samples"][sample_id]), "target"]
                        y_test = y_test.values.flatten()

                        #ajustando o target y para o valor correspondente a saída do circuito [-1, 1], classe 0 = -1 e classe 1 = 1
                        y_train = 2*y_train - 1
                        y_test = 2*y_test - 1

                        # Treinamento do modelo
                        for i in tqdm(range(epoch), desc=f"Samples id: {sample_id}, Treinando o modelo", unit="epoch"):

                            params,_,_,_,bias = opt.step(cost_function, params, X_train, y_train, ansatz_id, bias, amplitde_embedding=amplitde_embedding)
                            
                            if i == epoch - 1:
                                # Avaliando a acurácia no final do treinamento
                                accuracy_train = accuracy(params, X_train, y_train, ansatz_id, bias, amplitde_embedding=amplitde_embedding)
                                print(f"Epoch {i+1}/{epoch}, Acurácia de Treinamento: {accuracy_train:.4f}")

                        # Avaliando a acurácia no conjunto de teste
                        accuracy_test = accuracy(params, X_test, y_test, ansatz_id, bias, amplitde_embedding=amplitde_embedding)
                        print(f"Acurácia de Teste: {accuracy_test:.4f}")


                        # Salvando os resultados em um dicionário
                        results = {
                            "TIPO_DE_CARREGAMENTO_ENTRADA": amplitde_embedding,
                            "ARQUITETURA_ANSATZ": ansatz_id,
                            "OTIMIZADOR": opt.__class__.__name__,
                            "BASE_DE_DADOS": df.nome,
                            "SAMPLE": sample_id,
                            "METRICA_NAO_SUPERVISIONADA": cost_function.__name__,
                            "INDICE_DIVISAO_BASE_DE_DADOS": 0.7,
                            "ACURACIA_TREINAMENTO": accuracy_train,
                            "ACURACIA_TESTE": accuracy_test
                        }

                        # Salvando os resultados em um csv
                        salvar_dataframe(pd.DataFrame([results]), "./../ansatz_result/resultados.csv")

"""

'\nrng_weights = np.random.default_rng(42)\n\nfor ansatz_id in ansatz_ids:\n    for (sample, df) in (zip(samples, data_frames)):\n        for sample_id in samples_ids:\n            for cost_function in (cost_functions):\n                for  opt in (optimizers):\n                    for amplitde_embedding in ([True]):\n\n                        # resetando o optimizador\n                        if opt.__class__.__name__ == "GradientDescentOptimizer":\n                            opt = qml.GradientDescentOptimizer(stepsize=0.1)\n                        elif opt.__class__.__name__ == "AdamOptimizer":\n                            opt = qml.AdamOptimizer(stepsize=0.1)\n\n                        # Definindo os parâmetros atualizaveis pelo otimizador\n                        num_params = 3 if ansatz_id not in [0, 3, 9, 11, 13, 15, 18, 21, 26, 28] else 2\n                        params = np.array(2 * rng_weights.random(num_params) * np.pi, requires_grad=True)\n                        bias = n

In [497]:
def train_wrapper(args):
    """Função que encapsula toda a lógica de treinamento para paralelização"""
    (ansatz_id, sample, df, sample_id, cost_function, opt_class, 
     amplitde_embedding, rng_weights, nome,save_path) = args
    
    # Reset do otimizador
    if opt_class.__class__.__name__ == "GradientDescentOptimizer":
        opt = qml.GradientDescentOptimizer(stepsize=0.1)
    elif opt_class.__class__.__name__ == "AdamOptimizer":
        opt = qml.AdamOptimizer(stepsize=0.1)
    else:
        opt = opt_class(stepsize=0.1)
    
    # Configuração dos parâmetros
    num_params = 3 if ansatz_id not in [0, 3, 9, 11, 13, 15, 18, 21, 26, 28] else 2
    params = np.array(2 * rng_weights.random(num_params) * np.pi, requires_grad=True)
    bias = np.array(0.0, requires_grad=True)

    # Preparação dos dados
    X_train = df.loc[sample["samples"][sample_id], df.columns != "target"].to_numpy()
    y_train = 2*df.loc[sample["samples"][sample_id], "target"].values.flatten() - 1
    
    X_test = df.loc[~df.index.isin(sample["samples"][sample_id]), df.columns != "target"].to_numpy()
    y_test = 2*df.loc[~df.index.isin(sample["samples"][sample_id]), "target"].values.flatten() - 1

    # Treinamento
    epoch = 20
    for i in range(epoch):
        params,_,_,_,bias = opt.step(cost_function, params, X_train, y_train, ansatz_id, bias, 
                               amplitde_embedding=amplitde_embedding)
    
    # Avaliação
    accuracy_train = accuracy(params, X_train, y_train, ansatz_id, bias, 
                             amplitde_embedding=amplitde_embedding)
    accuracy_test = accuracy(params, X_test, y_test, ansatz_id, bias, 
                            amplitde_embedding=amplitde_embedding)

    # Retorna resultados para coleta
    return {
        "TIPO_DE_CARREGAMENTO_ENTRADA": amplitde_embedding,
        "ARQUITETURA_ANSATZ": ansatz_id,
        "OTIMIZADOR": opt.__class__.__name__,
        "BASE_DE_DADOS": nome,
        "SAMPLE": sample_id,
        "METRICA_NAO_SUPERVISIONADA": cost_function.__name__,
        "INDICE_DIVISAO_BASE_DE_DADOS": 0.7,
        "ACURACIA_TREINAMENTO": accuracy_train,
        "ACURACIA_TESTE": accuracy_test
    }

In [498]:
# Configuração de paralelização
def parallel_training(ansatz_ids, samples, data_frames, samples_ids, 
                     cost_functions, optimizers, save_path, max_workers=None):
    
    rng_weights = np.random.default_rng(42)
    tasks = []
    
    # Gerar todas as combinações de parâmetros
    for ansatz_id in ansatz_ids:
        for sample, df in zip(samples, data_frames):
            for sample_id in samples_ids:
                for cost_function in cost_functions:
                    for opt_class in optimizers:
                        for amplitde_embedding in [True]:
                            tasks.append((
                                ansatz_id, sample, df, sample_id, 
                                cost_function, opt_class, amplitde_embedding, 
                                rng_weights, df.nome, save_path
                            ))

    # Execução paralela
    with concurrent.futures.ProcessPoolExecutor(max_workers=max_workers) as executor:
        results = list(tqdm(executor.map(train_wrapper, tasks), 
                      total=len(tasks), 
                      desc="Executando Experimentos"))
    
    # Salvar resultados
    final_df = pd.DataFrame(results)
    salvar_dataframe(final_df, save_path)

In [ ]:
# Uso
parallel_training(

    ansatz_ids = list(range(30)),

    samples = [
    metrics1, metrics2, metrics3,
    metrics4, metrics5, metrics6,
    metrics7, metrics8, metrics9,
    metrics10, metrics11, metrics12
    ],

    data_frames = [
        df1, df2, df3, df4, df5, df6,
        df7, df8, df9, df10, df11, df12
    ],

    samples_ids = list(range(30)),

    cost_functions = [MSE],

    optimizers = [qml.AdamOptimizer(stepsize=0.1)],

    save_path = "./../ansatz_result/resultados.csv",

    max_workers = 12 
)

Executando Experimentos: 100%|██████████| 10800/10800 [16:25:37<00:00,  5.48s/it]  
